In [ ]:
!pip install koreanize-matplotlib #한글폰트 라이브러리 설치
!apt-get install -y fonts-nanum #나눔 고딕 폰트 설치
!pip install -q gdown

import gdown
import matplotlib.pyplot as plt
import koreanize_matplotlib  # 한글 자동 설정됨
import pandas as pd

from google.colab import drive

drive.mount('/content/drive')

pd.set_option('display.max_rows', None)
# plt.rcParams['font.family'] = 'NanumGothic' #나눔 고딕 폰트 설정

In [ ]:
# CSV 파일 불러오기

transactions_file_id = '1rzQ8Bz6hGQ7IYZPWzEkaXIJXvA6pCFkq'
output_path = 'transactions_hm.csv'
gdown.download(f'https://drive.google.com/uc?id={transactions_file_id}', output_path, quiet=False)


articles = pd.read_csv('https://drive.google.com/uc?id=1OYMa1QBf6Lt2vVABrFJi8FO4UFhieVRv')
customer = pd.read_csv('https://drive.google.com/uc?id=1uyHiU-iKwJi8ht3TqBqmmFojvSc0CY_N')
transactions = pd.read_csv('transactions_hm.csv')

In [ ]:
# 각 테이블에 대한 열과 행 개수를 표현해주세요.
articles.shape # (105542, 25)
customer.shape # (1048575, 6)
transactions.shape # (1048575, 5)


In [ ]:
# 각 테이블에 대한 컬럼타입과 통계량을 보여주세요.
articles.info()
articles.describe()
customer.info()
customer.describe()
transactions.info()
transactions.describe()

In [ ]:
# 테이블을 결합하여, 데이터분석을 위한 하나의 데이터셋으로 만들어주세요.
# 모든 테이블을 결합하지 않아도 좋습니다.
# 테이블 결합 시 inner join 으로 진행해주세요.
df_total_table = pd.merge(transactions, customer, on='customer_id', how='inner')
df_total_table = pd.merge(df_total_table, articles, on='article_id', how='inner')
df_total_table.head()

In [ ]:
# NULL 값이 존재하는 경우, 대체 or 제거 모두 가능합니다.
articles.isnull().sum() # 제품 상세 설명 null 이므로 대체/제거 하지 않음
customer.isnull().sum() # 뉴스 레터 주기 null 값이나 큰 영향을 주지 않아 대체/제거 하지 않음.
transactions.isnull().sum() # null 값 없음

In [ ]:
df_total_table["t_dat"].head()

In [ ]:
df_total_table["t_datetime"] = pd.to_datetime(df_total_table["t_dat"])
df_total_table["month"] = df_total_table["t_datetime"].dt.month

# 실제 금액처럼 보이게 price 정규화 해제 (예: 100,000 곱하기)
df_total_table["price_real"] = df_total_table["price"] * 100000

# 월 별 판매량과 주문 수량 집계
df_total_table.groupby("month").agg(
    total_sales=("price_real", "sum"),
    order_count=("price", "count")
).sort_index()

# 월 별 통계량
# df_total_table.groupby("month")["price"].agg(["mean", "median", "max", "min", "count"])

In [ ]:
import matplotlib.pyplot as plt

# 월별 집계
monthly_summary = df_total_table.groupby("month").agg(
    total_sales=("price", "sum"),
    order_count=("price", "count")
).sort_index()

# 통계값 계산
mean_sales = monthly_summary['total_sales'].mean()
median_sales = monthly_summary['total_sales'].median()
mean_orders = monthly_summary['order_count'].mean()
median_orders = monthly_summary['order_count'].median()

# 시각화 시작
fig, ax1 = plt.subplots(figsize=(12, 6))

# ① 실선: 판매 매출
ax1.plot(
    monthly_summary.index,
    monthly_summary['total_sales'],
    color='tab:blue',
    marker='o',
    linestyle='-',
    label='Total Sales'
)
# 평균선 & 중앙값선 (점선)
ax1.axhline(mean_sales, color='tab:blue', linestyle=':', label='Sales Mean')
ax1.axhline(median_sales, color='tab:blue', linestyle='-.', label='Sales Median')

ax1.set_xlabel('Month')
ax1.set_ylabel('Total Sales (₩)', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.set_xticks(monthly_summary.index)

# ② 실선: 주문량
ax2 = ax1.twinx()
ax2.plot(
    monthly_summary.index,
    monthly_summary['order_count'],
    color='tab:red',
    marker='s',
    linestyle='-',
    label='Order Count'
)
# 평균선 & 중앙값선 (점선)
ax2.axhline(mean_orders, color='tab:red', linestyle=':', label='Order Mean')
ax2.axhline(median_orders, color='tab:red', linestyle='-.', label='Order Median')

ax2.set_ylabel('Order Count', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

# 제목
plt.title('월별 판매 매출 & 주문량 (평균선/중앙값 포함)')
fig.tight_layout()

# 범례 병합
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
fig.legend(lines1 + lines2, labels1 + labels2, loc='upper center', ncol=3)

plt.show()


In [ ]:
# 월 별 색상 인기도 범프차트

# ② 월-색상별 count 계산
counts = df_total_table.groupby("month")["colour_group_name"].value_counts()
counts = counts.rename("count").reset_index()

# ③ 월별 상위 10개 색상만 추출
top10_per_month = counts.groupby("month").apply(
    lambda x: x.nlargest(10, 'count')
).reset_index(drop=True)

# ④ pivot + 순위 매기기
pivot_df = top10_per_month.pivot_table(
    index="colour_group_name",
    columns="month",
    values="count",
    fill_value=0
)

# 순위로 변환 (높을수록 낮은 순위 → 반전)
# 0값을 NaN으로 변환
pivot_df = pivot_df.replace(0, pd.NA)

# 순위 계산 (NaN은 가장 낮은 순위로 처리)
pivot_df = pivot_df.rank(ascending=False, method='first', na_option='bottom')

# 10위 초과 순위는 NaN 처리해서 그래프에서 제외
pivot_df = pivot_df.where(pivot_df <= 10)


# ⑤ 범프 차트 그리기
plt.figure(figsize=(14, 8))

for color_name in pivot_df.index:
    ranks = pivot_df.loc[color_name]
    plt.plot(
        ranks.index, ranks.values,
        marker='o', linestyle='-', label=color_name
    )

    # 각 점에 순위 텍스트 표시
    for month, rank in ranks.items():
        if pd.notna(rank):
            plt.text(month, rank - 0.15, str(int(rank)),
                     ha='center', va='bottom', fontsize=9)

# y축 뒤집기: 1등이 위로 오도록
plt.gca().invert_yaxis()

# x축: 월
plt.xticks(range(1, 13))
plt.gca().xaxis.set_ticks_position('bottom')
plt.gca().yaxis.set_ticks_position('none')
plt.gca().spines['bottom'].set_position(('outward', 40))
plt.gca().set_ylim(10.5, 0.5)

plt.xlabel('Month')
plt.ylabel('Rank')
plt.title('월별 Top 10 색상 범프차트')
plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
plt.grid(axis='y')

plt.show()


In [ ]:
# 시즌 별 컬러 인기도 범프 차트

def get_season(month):
    if month in [3, 4, 5]:
        return '봄'
    elif month in [6, 7, 8]:
        return '여름'
    elif month in [9, 10, 11]:
        return '가을'
    elif month in [12, 1, 2,]:
        return '겨울'
df_total_table['season'] = df_total_table['month'].apply(get_season)
# 1. season별 색상별 count 계산
season_color_counts = df_total_table.groupby("season")["colour_group_name"].value_counts()

# 2. Series → DataFrame 변환 후 인덱스 리셋
season_color_counts = season_color_counts.rename("count").reset_index()

# 3. season별 상위 10개 색상만 추출
top10_per_season = season_color_counts.groupby("season").apply(lambda x: x.nlargest(10, "count")).reset_index(drop=True)


# 순위 계산
top10_per_season['rank'] = top10_per_season.groupby('season')['count'].rank(ascending=False, method='first')

# season 순서 정렬 (한글이라 수동 정렬 필요)
season_order = ['봄', '여름', '가을', '겨울']
top10_per_season['season'] = pd.Categorical(top10_per_season['season'], categories=season_order, ordered=True)

# pivot: 색상별로 season별 순위 배열
pivot_df = top10_per_season.pivot(index='colour_group_name', columns='season', values='rank')

# 범프 차트 그리기
plt.figure(figsize=(10, 6))

for color_name in pivot_df.index:
    ranks = pivot_df.loc[color_name]
    plt.plot(season_order, ranks, marker='o', label=color_name)
    for season, rank in ranks.items():
        if pd.notna(rank):
            plt.text(season, rank - 0.15, str(int(rank)), ha='center', va='bottom', fontsize=9)

plt.gca().invert_yaxis()
plt.ylim(10.5, 0.5)
plt.xlabel('Season')
plt.ylabel('Rank')
plt.title('Top 10 Colors by Season')
plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1))
plt.grid(axis='y')

plt.show()


In [ ]:
df_total_table.groupby("month")["graphical_appearance_name"].value_counts()

In [ ]:
# 2. 월-graphical_appearance_name 별 개수 계산
counts = df_total_table.groupby("month")["graphical_appearance_name"].value_counts()
counts = counts.rename("count").reset_index()

# 3. 월별 상위 10개만 추출
top10_per_month = counts.groupby("month").apply(
    lambda x: x.nlargest(10, 'count')
).reset_index(drop=True)

# 4. 피벗 테이블 생성 (index: appearance name, columns: month)
pivot_df = top10_per_month.pivot_table(
    index="graphical_appearance_name",
    columns="month",
    values="count",
    fill_value=0
)

# 5. 0을 NaN으로 바꿔서 랭킹에서 제외
pivot_df = pivot_df.replace(0, pd.NA)

# 6. 월별 순위 계산 (내림차순, NaN은 아래)
pivot_df = pivot_df.rank(ascending=False, method='first', na_option='bottom')

# 7. 10위 초과는 NaN으로 처리
pivot_df = pivot_df.where(pivot_df <= 10)

# 8. 범프 차트 그리기
plt.figure(figsize=(14, 8))

for appearance_name in pivot_df.index:
    ranks = pivot_df.loc[appearance_name]
    plt.plot(
        ranks.index, ranks.values,
        marker='o', linestyle='-', label=appearance_name
    )

    # 순위 텍스트 표시
    for month, rank in ranks.items():
        if pd.notna(rank):
            plt.text(month, rank - 0.15, str(int(rank)),
                     ha='center', va='bottom', fontsize=9)

# y축 반전 (1등이 위)
plt.gca().invert_yaxis()

plt.xticks(range(1, 13))
plt.gca().xaxis.set_ticks_position('bottom')
plt.gca().yaxis.set_ticks_position('none')
plt.gca().spines['bottom'].set_position(('outward', 40))
plt.gca().set_ylim(10.5, 0.5)

plt.xlabel('Month')
plt.ylabel('Rank')
plt.title('월별 Top 10 Graphical Appearance Bump Chart')
plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
plt.grid(axis='y')

plt.show()

In [ ]:
df_total_table.groupby("month")["perceived_colour_value_id"].value_counts()

In [ ]:
counts = df_total_table.groupby("month")["perceived_colour_value_id"].value_counts()
counts = counts.rename("count").reset_index()

# 3. 월별 상위 10개만 추출
top10_per_month = counts.groupby("month").apply(
    lambda x: x.nlargest(10, 'count')
).reset_index(drop=True)

# 4. 피벗 테이블 생성
pivot_df = top10_per_month.pivot_table(
    index="perceived_colour_value_id",
    columns="month",
    values="count",
    fill_value=0
)

# 5. 0을 NaN 처리
pivot_df = pivot_df.replace(0, pd.NA)

# 6. 순위 계산 (내림차순, NaN은 가장 아래)
pivot_df = pivot_df.rank(ascending=False, method='first', na_option='bottom')

# 7. 10위 초과는 NaN 처리
pivot_df = pivot_df.where(pivot_df <= 10)

# 8. 범프 차트 그리기
plt.figure(figsize=(14, 8))

for value_id in pivot_df.index:
    ranks = pivot_df.loc[value_id]
    plt.plot(
        ranks.index, ranks.values,
        marker='o', linestyle='-', label=str(value_id)
    )

    # 순위 텍스트 표시
    for month, rank in ranks.items():
        if pd.notna(rank):
            plt.text(month, rank - 0.15, str(int(rank)),
                     ha='center', va='bottom', fontsize=9)

plt.gca().invert_yaxis()

plt.xticks(range(1, 13))
plt.gca().xaxis.set_ticks_position('bottom')
plt.gca().yaxis.set_ticks_position('none')
plt.gca().spines['bottom'].set_position(('outward', 40))
plt.gca().set_ylim(10.5, 0.5)

plt.xlabel('Month')
plt.ylabel('Rank')
plt.title('월별 Top 10 명도 (perceived_colour_value_id) 범프 차트')
plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
plt.grid(axis='y')

plt.show()

In [ ]:
df_total_table.groupby("month")["garment_group_name"].value_counts()

In [ ]:
# 2. 월별 garment_group_name별 개수 계산
counts = df_total_table.groupby("month")["garment_group_name"].value_counts()
counts = counts.rename("count").reset_index()

# 3. 월별 상위 10개만 추출
top10_per_month = counts.groupby("month").apply(
    lambda x: x.nlargest(10, 'count')
).reset_index(drop=True)

# 4. 피벗 테이블 생성 (index: garment_group_name, columns: month)
pivot_df = top10_per_month.pivot_table(
    index="garment_group_name",
    columns="month",
    values="count",
    fill_value=0
)

# 5. 0을 NaN으로 변환
pivot_df = pivot_df.replace(0, pd.NA)

# 6. 월별 순위 계산 (내림차순, NaN은 가장 낮은 순위로)
pivot_df = pivot_df.rank(ascending=False, method='first', na_option='bottom')

# 7. 10위 초과는 NaN 처리
pivot_df = pivot_df.where(pivot_df <= 10)

# 8. 범프 차트 그리기
plt.figure(figsize=(14, 8))

for garment in pivot_df.index:
    ranks = pivot_df.loc[garment]
    plt.plot(
        ranks.index, ranks.values,
        marker='o', linestyle='-', label=garment
    )
    # 순위 텍스트 표시
    for month, rank in ranks.items():
        if pd.notna(rank):
            plt.text(month, rank - 0.15, str(int(rank)),
                     ha='center', va='bottom', fontsize=9)

plt.gca().invert_yaxis()

plt.xticks(range(1, 13))
plt.gca().xaxis.set_ticks_position('bottom')
plt.gca().yaxis.set_ticks_position('none')
plt.gca().spines['bottom'].set_position(('outward', 40))
plt.gca().set_ylim(10.5, 0.5)

plt.xlabel('Month')
plt.ylabel('Rank')
plt.title('월별 Top 10 Garment Group 범프 차트')
plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
plt.grid(axis='y')

plt.show()

In [ ]:
# 2. 세 컬럼 조합해서 새로운 그룹명 생성
df_total_table["combo_group"] = (
    df_total_table["colour_group_name"].astype(str) + " / " +
    df_total_table["graphical_appearance_name"].astype(str) + " / " +
    df_total_table["garment_group_name"].astype(str)
)
433
# 3. 월-조합별 개수 계산
counts = df_total_table.groupby("month")["combo_group"].value_counts()
counts = counts.rename("count").reset_index()

# 4. 월별 Top 10만 추출
top10_per_month = counts.groupby("month").apply(
    lambda x: x.nlargest(10, 'count')
).reset_index(drop=True)

# 5. 피벗 테이블 생성
pivot_df = top10_per_month.pivot_table(
    index="combo_group",
    columns="month",
    values="count",
    fill_value=0
)

# 6. 0 -> NaN 변환
pivot_df = pivot_df.replace(0, pd.NA)

# 7. 순위 계산 (내림차순, NaN은 아래)
pivot_df = pivot_df.rank(ascending=False, method='first', na_option='bottom')

# 8. 10위 초과는 NaN 처리
pivot_df = pivot_df.where(pivot_df <= 10)

# 9. 범프 차트 그리기
plt.figure(figsize=(16, 10))

for combo in pivot_df.index:
    ranks = pivot_df.loc[combo]
    plt.plot(
        ranks.index, ranks.values,
        marker='o', linestyle='-', label=combo
    )
    # 순위 텍스트 표시
    for month, rank in ranks.items():
        if pd.notna(rank):
            plt.text(month, rank - 0.15, str(int(rank)),
                     ha='center', va='bottom', fontsize=8)

plt.gca().invert_yaxis()

plt.xticks(range(1, 13))
plt.gca().xaxis.set_ticks_position('bottom')
plt.gca().yaxis.set_ticks_position('none')
plt.gca().spines['bottom'].set_position(('outward', 40))
plt.gca().set_ylim(10.5, 0.5)

plt.xlabel('Month')
plt.ylabel('Rank')
plt.title('월별 Top 10 Colour / Appearance / Garment Group Combo Bump Chart')
plt.legend(loc='upper right', bbox_to_anchor=(1.25, 1), fontsize=8)
plt.grid(axis='y')

plt.show()

In [ ]:
# 기준컬럼을 두고 집계함수를 사용하여 비교분석을 진행해주세요. 그리고 이를 시각화 해주세요.
# 최소 1개의 인사이트를 제시해야 합니다.